## IBM Quantum Challenge Fall 2021

# Challenge 3: 양자 기계 학습을 사용한 이미지를 분류

<div id='problem'></div>
<div class="alert alert-block alert-info">
    
최고의 경험을 위해 오른쪽 상단의 계정 메뉴에서 **light** 워크스페이스 테마로 전환하는 것을 추천합니다.</div>

## 소개

머신 러닝은 뛰어난 성능과 다재다능함으로 많은 주목을 받고 있는 기술입니다. 최근 알고리즘의 발달과 연산 자원의 증가로 많은 산업 분야에서 실제로 활용되고 있습니다. 한가지 예는 머신 러닝이 이제 인간과 같거나 더 나은 정확도로 이미지를 분류할 수 있는 컴퓨터 비전 분야입니다. 예를 들어, 옷들의 이미지를 자동으로 분류하는 기술은 온라인 의류 쇼핑을 더욱 편리하게 만들어 주었습니다.

양자 연산을 기계 학습에 적용하는 것은 최근의 연구결과들을 통해 훨씬 더 큰 능력을 가질 가능성이 있는 것으로 나타났습니다. 양자 서포트 벡터 머신(QSVM)과 양자 생성적 적대 신경망(QGAN)와 같은 양자 머신러닝을 위한 다양한 알고리즘이 제안되었습니다. 이 연습에서는 의류 이미지 분류 작업에 QSVM을 사용해 보도록 하겠습니다.

QSVM은 고전적인 기계 학습 알고리즘인 서포트 벡터 머신(SVM)의 양자 버전입니다. QSVM의 다양한 접근법 중, 일부는 내결함성 양자 컴퓨터를 가정하여 계산을 가속화하는 것을 목표로 하는 반면, 다른 일부는 노이즈가 많은 현재의 장치를 가정하여 더 높은 표현력을 달성하는 것을 목표로 합니다. 이 연습에서는 후자에 초점을 맞추고 자세한 내용은 이어서 다루도록 하겠습니다.

QSVM 구현을 위해서, 양자 모델을 구성하는 방법을 고를 수 있는데, 특히 양자 특징 맵(feature map)에 더욱 주목할 수 있습니다. 더 복잡한 특징 맵은 표현력은 높지만 노이즈가 많은 현재의 장치를 사용할 때 주의해야할 노이즈에 더 민감한 특성을 지니게 되는 트레이드-오프를 갖습니다

이 도전 과제에 등장하는 많은 개념들은 2021 QGSS(Qiskit Global Summer School)에서 이미 설명되었습니다. 여름학교의 강의 자료와 강의 영상을 함께 공부해 보십시오. 각 파트에 남겨진 해당 강의들의 링크를 참고하세요.

<center><div><img src="./resources/ecommerce.jpg" width="640" /></div></center>

## 도전 과제
<div class="alert alert-block alert-success">

<b>목표</b> 

다중 클래스 분류를 위한 QSVM 모델을 구현하고 레이블을 정확하게 예측합니다.    
    
<b>계획</b> 

먼저 간단한 데이터 세트의 이진 분류를 위한 QSVM을 구성하는 방법에 대해 알아봅니다. 그 다음 학습한 내용을 다른 데이터 세트의 더 복잡한 3-클래스 분류에 적용합니다.

**1. 튜토리얼 - MNIST의 이진 분류를 위한 QSVM:** QSVM의 일반적인 작업 흐름에 익숙해지고 차원/특징(dimension/feature) 맵의 최적의 조합을 찾습니다.

**2. 도전 - Fashion-MNIST의 3-클래스 분류를 위한 QSVM:** 이진 QSVM 분류기를 사용하여 3-class 분류기를 구현합니다. 차원/특징 맵의 최상의 조합을 찾기 위해 첫 번째 단계와 유사한 과정을 수행합니다. 더 작은 특징 맵 회로로 더 나은 정확도를 달성해 보십시오.
</div>


<div class="alert alert-block alert-info">
    
시작하기에 앞서,[**Qiskit Machine Learning Demo Session with Anton Dekusar**](https://youtu.be/claoY57eVIc?t=1814)을 시청하고 데모에 사용된[**demo notebook**](https://github.com/qiskit-community/qiskit-application-modules-demo-sessions/tree/main/qiskit-machine-learning) 
    
</div>

In [ ]:
# General imports
import os
import gzip
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
import warnings

warnings.filterwarnings("ignore")

# scikit-learn imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Qiskit imports
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel

## Part 1: 튜토리얼 - MNIST의 이진 분류를 위한 QSVM

이 파트에서는 손으로 쓴 숫자 4와 9의 이진 분류에 QSVM을 적용해 보겠습니다. 이 튜토리얼을 통해 QSVM을 이진 분류에 적용하는 작업 흐름을 배우게 됩니다. 더 나은 조합을 찾아 더 높은 정확도를 달성해 봅시다.

관련 QGSS 자료:
- Lab 3: https://www.youtube.com/watch?v=GVhCOTzAkCM&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=17

### 1. 데이터 준비

처음 작업에 사용할 데이터는 공공에 개방되어 사용할 수 있는 유명한 필기 숫자 데이터의 작은 하위 집합입니다. 우리는 '4'와 '9'를 구별하는 것을 목표로 합니다.

데이터셋에는 총 100개의 데이터가 있습니다. 이 중 80개는 레이블이 지정된 학습 데이터이고 나머지 20개는 레이블이 지정되지 않은 테스트 데이터입니다. 각 데이터는 배열로 압축된 28x28 정수 이미지로, 각 요소는 0(흰색)에서 255(검정색) 사이의 정수값을 갖습니다. 양자 분류에 데이터 세트를 사용하려면 범위를 -1과 1 사이로 조절하고 데이터의 차원을 사용하려는 큐비트 수로 줄여야 합니다(이 문제의 N_DIM=5).

In [ ]:
# Load MNIST dataset
DATA_PATH = './resources/ch3_part1.npz'
data = np.load(DATA_PATH)

sample_train = data['sample_train']
labels_train = data['labels_train']
sample_test = data['sample_test']

# Split train data
sample_train, sample_val, labels_train, labels_val = train_test_split(
    sample_train, labels_train, test_size=0.2, random_state=42)

# Visualize samples
fig = plt.figure()

LABELS = [4, 9]
num_labels = len(LABELS)
for i in range(num_labels):
    ax = fig.add_subplot(1, num_labels, i+1)
    img = sample_train[labels_train==LABELS[i]][0].reshape((28, 28))
    ax.imshow(img, cmap="Greys")

In [ ]:
# Standardize
ss = StandardScaler()
sample_train = ss.fit_transform(sample_train)
sample_val = ss.transform(sample_val)
sample_test = ss.transform(sample_test)

# Reduce dimensions
N_DIM = 5
pca = PCA(n_components=N_DIM)
sample_train = pca.fit_transform(sample_train)
sample_val = pca.transform(sample_val)
sample_test = pca.transform(sample_test)

# Normalize
mms = MinMaxScaler((-1, 1))
sample_train = mms.fit_transform(sample_train)
sample_val = mms.transform(sample_val)
sample_test = mms.transform(sample_test)

### 2. 데이터 인코딩(Data Encoding)

우리는 고전적인 데이터를 양자 특징 맵을 이용하여 양자 상태 공간으로 인코딩할 것입니다. 특징 맵을 분류하려는 데이터 세트에 따라 적절하게 선택하는 것이 매우 중요합니다. 여기서는 Qiskit에서 사용할 수 있는 특징 맵의 종류를 살펴본 후 데이터를 인코딩할 특징 맵을 선택하고 우리의 데이터를 인코딩 하기 위해 수정하도록 하겠습니다.

### 2.1 양자 특징 맵(Quantum Feature Maps)
이름에서 알 수 있듯이, 양자 특징 맵 $\phi(\mathbf{x})$은 고전적 특징 벡터 $\mathbf{x}$를 양자 상태 $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$로 매핑합니다. 이것은 _n_ 이 인코딩에 사용되는 큐비트의 개수일 때, 초기 상태 $|0\rangle^{n}$에 유니터리 연산 $\mathcal{U}_{\Phi(\mathbf{x})}$을 적용함으로써 얻어집니다.

현재 Qiskit에서 사용할 수 있는 다음의 특징 맵은 [**_Havlicek et al_.  Nature **567**, 209-212 (2019)**](https://www.nature.com/articles/s41586-019-0980-2) 에 소개된 것으로, 특히 `ZZFeatureMap`은 고전적인 방법으로 시뮬레이션하기 어려운 것으로 예상되지만, 현재의 양자 장치에서 짧은 양자 회로로 구현할 수 있습니다.

- [**`PauliFeatureMap`**](https://qiskit.org/documentation/stubs/qiskit.circuit.library.PauliFeatureMap.html)
- [**`ZZFeatureMap`**](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZFeatureMap.html)
- [**`ZFeatureMap`**](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html)

`PauliFeatureMap`은 다음과 같이 정의되며:

```python
PauliFeatureMap(feature_dimension=None, reps=2, 
                entanglement='full', paulis=None, 
                data_map_func=None, parameter_prefix='x',
                insert_barriers=False)
```

$d$의 회로 깊이를 갖는 유니터리 연산자를 표현합니다:

$$ \mathcal{U}_{\Phi(\mathbf{x})}=\prod_d U_{\Phi(\mathbf{x})}H^{\otimes n},\ U_{\Phi(\mathbf{x})}=\exp\left(i\sum_{S\subseteq[n]}\phi_S(\mathbf{x})\prod_{k\in S} P_i\right), $$

이 연산자는 얽힘 블록에 하다마드 게이트를 끼워넣은 레이어를 포함하며, $U_{\Phi(\mathbf{x})}$, 고전적인 데이터를 인코딩하고, $d=2$일 때의 회로도는 아래와 같습니다.

<center><img src="./resources/featuremap.png" width="1000" /></center>

얽힌 블록들 속에서, $U_{\Phi(\mathbf{x})}$: $P_i \in \{ I, X, Y, Z \}$는 파울리 행렬을 의미하고, $S$ 인텍스는 서로 다른 큐비트들과 데이터 포인트들 간의 연결성을 나타냅니다: $S \in \{\binom{n}{k}\ combinations,\ k = 1,... n \}$, 그리고 기본 설정으로 데이터 매핑 함수 $\phi_S(\mathbf{x})$ 는 
$$\phi_S:\mathbf{x}\mapsto \Bigg\{\begin{array}{ll}
    x_i & \mbox{if}\ S=\{i\} \\
        (\pi-x_i)(\pi-x_j) & \mbox{if}\ S=\{i,j\}
    \end{array}$$

이때 $k = 1, P_0 = Z$이고, 이것이 `ZFeatureMap`: 
$$\mathcal{U}_{\Phi(\mathbf{x})} = \left( \exp\left(i\sum_j \phi_{\{j\}}(\mathbf{x}) \, Z_j\right) \, H^{\otimes n} \right)^d.$$

으로 아래와 같이 정의됩니다:
```python
ZFeatureMap(feature_dimension, reps=2, 
            data_map_func=None, insert_barriers=False)
```

In [ ]:
# 3 features, depth 2
map_z = ZFeatureMap(feature_dimension=3, reps=2)
map_z.decompose().draw('mpl')

이 특징 맵에 얽힘이 없다는 점을 주목합시다. 이것은 이 특징 맵이 고전적으로 시뮬레이션하기 쉽고 양자 이점을 제공하지 않는다는 것을 뜻합니다

또한 $k = 2, P_0 = Z, P_1 = ZZ$라면, 이것은 `ZZFeatureMap`이 되고: 
$$\mathcal{U}_{\Phi(\mathbf{x})} = \left( \exp\left(i\sum_{jk} \phi_{\{j,k\}}(\mathbf{x}) \, Z_j \otimes Z_k\right) \, \exp\left(i\sum_j \phi_{\{j\}}(\mathbf{x}) \, Z_j\right) \, H^{\otimes n} \right)^d.$$ 

다음과 같이 정의 됩니다:
```python
ZZFeatureMap(feature_dimension, reps=2, 
             entanglement='full', data_map_func=None, 
             insert_barriers=False)
```

In [ ]:
# 3 features, depth 1, linear entanglement
map_zz = ZZFeatureMap(feature_dimension=3, reps=1, entanglement='linear')
map_zz.decompose().draw('mpl')

이 특징 맵은 얽힘을 가지고 있다는 것에 주목합시다, 얽힘 맵은 다음과 같이 정의 할 수 있습니다:

In [ ]:
# 3 features, depth 1, circular entanglement
map_zz = ZZFeatureMap(feature_dimension=3, reps=1, entanglement='circular')
map_zz.decompose().draw('mpl')

특징 맵에서 파울리 게이트들을 사용자 정의 할수도 있습니다, 예를 들어, $P_0 = X, P_1 = Y, P_2 = ZZ$ 라면:
$$\mathcal{U}_{\Phi(\mathbf{x})} = \left( \exp\left(i\sum_{jk} \phi_{\{j,k\}}(\mathbf{x}) \, Z_j \otimes Z_k\right) \, \exp\left(i\sum_{j} \phi_{\{j\}}(\mathbf{x}) \, Y_j\right) \, \exp\left(i\sum_j \phi_{\{j\}}(\mathbf{x}) \, X_j\right) \, H^{\otimes n} \right)^d.$$ 

In [ ]:
# 3 features, depth 1
map_pauli = PauliFeatureMap(feature_dimension=3, reps=1, paulis = ['X', 'Y', 'ZZ'])
map_pauli.decompose().draw('mpl')

Qiskit의 circuit library에 있는 [`NLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.NLocal.html) 과 [`TwoLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) 함수도 마찬가지로 매개 변수가 있는 양자 회로를 구성하는데 사용되어 특징 맵으로 사용할 수 있습니다.  

```python
TwoLocal(num_qubits=None, reps=3, rotation_blocks=None, 
         entanglement_blocks=None, entanglement='full',  
         skip_unentangled_qubits=False, 
         skip_final_rotation_layer=False, 
         parameter_prefix='θ', insert_barriers=False, 
         initial_state=None)
```

```python
NLocal(num_qubits=None, reps=1, rotation_blocks=None, 
       entanglement_blocks=None, entanglement=None,   
       skip_unentangled_qubits=False, 
       skip_final_rotation_layer=False, 
       overwrite_block_parameters=True, 
       parameter_prefix='θ', insert_barriers=False, 
       initial_state=None, name='nlocal')
```

두 함수 모두 번갈아 회전 및 얽힘 층이 나타나는 매개변수화된 회로를 만들어 냅니다. 각각의 레이어에서, 매개 변수화된 회로 블럭은 설계된 대로 양자 회로에서 동작합니다. 회전 레이어는 블록들이 서로 겹쳐서 적용되는 반면 얽힘 레이어에서는 얽힘 전략에 따라 구성 됩니다. 각 레이어는 여러 번 반복되며 일반적으로 최종 회전 레이어가 추가되어 마무리 됩니다.

`NLocal` 함수에서, 회로 블록은 임의의 크기(회로의 큐비트 수보다 작음)를 가질 수 있지만 `TwoLocal`에서는 회전 레이어는 모든 큐비트에 적용되는 싱글 큐비트 게이트로 구성되어 있고 얽힘 레이어는 2큐비트 게이트로 구성됩니다.

예를 들어, 아래의 `TwoLocal` 회로는, $R_y$와 $R_Z$ 게이트로 구성된 회전 레이어를 가지고 있으며 $CX$ 게이트가 얽힘 게이트에 사용되어 회전 얽힘(Circular entanglement)을 구현하였습니다.

In [ ]:
twolocal = TwoLocal(num_qubits=3, reps=2, rotation_blocks=['ry','rz'], 
               entanglement_blocks='cx', entanglement='circular', insert_barriers=True)
twolocal.decompose().draw('mpl')

그리고 상응하는 `NLocal` 회로는 다음과 같습니다:

In [ ]:
twolocaln = NLocal(num_qubits=3, reps=2,
               rotation_blocks=[RYGate(Parameter('a')), RZGate(Parameter('a'))], 
               entanglement_blocks=CXGate(), 
               entanglement='circular', insert_barriers=True)
twolocaln.decompose().draw('mpl')

그럼 이제 첫번째 학습 샘플을 `PauliFeatureMap` 을 사용해 인코드 해봅시다:

In [ ]:
print(f'First training data: {sample_train[0]}')

In [ ]:
encode_map = PauliFeatureMap(feature_dimension=N_DIM, reps=1, paulis = ['X', 'Y', 'ZZ'])
encode_circuit = encode_map.bind_parameters(sample_train[0])
encode_circuit.decompose().draw(output='mpl')

</div>
    
<div class="alert alert-block alert-success">

<b>도전 3a</b> 
    
3회 반복, '회전' 얽힘 및 나머지 옵션은 기본값을 사용하는 'ZZFeatureMap'을 사용하여 5차원으로 구성된 데이터를 인코딩하는 기능 맵을 구성합니다.
    
</div>

제출 양식:
```python
ex3a_fmap = ZZFeatureMap(...)
```

In [ ]:
##############################
# Provide your code here

ex3a_fmap = 



##############################

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex3a
grade_ex3a(ex3a_fmap)

### 2.2 양자 커널 추정 (Quantum Kernel Estimation)

양자 특징 맵 $\phi(\mathbf{x})$는 자연스럽게 양자 커널을 낳고 양자커널 $k(\mathbf{x}_i,\mathbf{x}_j)= \phi(\mathbf{x}_j)^\dagger\phi(\mathbf{x}_i)$는 유사성의 척도로 사용될 수 있습니다: $k(\mathbf{x}_i,\mathbf{x}_j)$는 $\mathbf{x}_i$와 $\mathbf{x}_j$가 유사할 수록 큰 값을 갖게 됩니다. 

데이터의 크기가 유한한 경우, 양자 커널은 행렬로 나타낼 수 있습니다: $K_{ij} = \left| \langle \phi^\dagger(\mathbf{x}_j)| \phi(\mathbf{x}_i) \rangle \right|^{2}$. 전이 진폭을 계산하여 양자 컴퓨터에서 커널 행렬의 각 요소를 계산할 수 있습니다:
$$
\left| \langle \phi^\dagger(\mathbf{x}_j)| \phi(\mathbf{x}_i) \rangle \right|^{2} = 
\left| \langle 0^{\otimes n} | \mathbf{U_\phi^\dagger}(\mathbf{x}_j) \mathbf{U_\phi}(\mathbf{x_i}) | 0^{\otimes n} \rangle \right|^{2}
$$
특징 맵이 매개변수화된 양자 회로라고 가정한다면, 이는 $n$ 큐비트에 대한 단일 유니터리 변환 $\mathbf{U_\phi}(\mathbf{x})$으로 설명될 수 있습니다. 

이것은 서포트 벡터 분류와 같은 커널 머신 러닝 알고리즘에서 사용할 수 있는 양자 커널 매트릭스의 추정치를 제공합니다.

[***Havlicek et al*.  Nature 567, 209-212 (2019)**](https://www.nature.com/articles/s41586-019-0980-2) 의 논문에서 논의된 바와 같이, 양자 커널 기계 알고리즘은 해당 양자 커널을 고전적으로 추정하기 어려운 경우에만 고전적인 접근법에 비해 양자 이점이 있습니다.

나중에 알게 되겠지만, 고전적인 자원으로 커널을 추정하는 것의 어려움은 양자적 이점을 얻기 위한 필요 조건이지만 언제나 만족되는 충분 조건은 아닙니다. 

하지만, 최근에 [***Liu et al.* arXiv:2010.02174 (2020)**](https://arxiv.org/abs/2010.02174) 의 논문에서 밝혀진 바와 같이 양자 커널 방법을 사용할때 모든 고전적 학습보다 양자적 이점을 갖는 학습 문제가 존재합니다.

훈련 및 테스트 데이터 세트가 준비된 상태에서, PauliFeatureMap으로 `QuantumKernel` 클래스를 설정하고 `BasicAer` `statevector_simulator`를 사용하여 훈련 및 테스트 커널 매트릭스를 추정해 봅시다.

In [ ]:
pauli_map = PauliFeatureMap(feature_dimension=N_DIM, reps=1, paulis = ['X', 'Y', 'ZZ'])
pauli_kernel = QuantumKernel(feature_map=pauli_map, quantum_instance=Aer.get_backend('statevector_simulator'))

학습 커널 행렬의 항목 중, 첫 번째 및 두 번째 학습 데이터 샘플 간의 전환 진폭을 계산해 봅시다.

In [ ]:
print(f'First training data : {sample_train[0]}')
print(f'Second training data: {sample_train[1]}')

우선, 회로를 생성하고 그려봅시다:

In [ ]:
pauli_circuit = pauli_kernel.construct_circuit(sample_train[0], sample_train[1])
pauli_circuit.decompose().decompose().draw(output='mpl')

게이트의 매개변수들은 읽기가 약간 어렵지만, 절반은 데이터 샘플 중 하나를 인코딩하고 다른 절반은 다른 하나를 인코딩하는 회로의 대칭적인 구조에 주목해 봅시다. 

다음으로 회로를 시뮬레이션합니다. 회로에 측정값이 포함되어 있으므로 `qasm_simulator`를 사용하지만 샘플링 노이즈의 영향을 줄이기 위해 샷 수를 늘립니다. 

In [ ]:
backend = Aer.get_backend('qasm_simulator')
job = execute(pauli_circuit, backend, shots=8192, 
              seed_simulator=1024, seed_transpiler=1024)
counts = job.result().get_counts(pauli_circuit)

전이 진폭은 전체 샷수 대비 "0" 상태 카운트 수의 비율입니다:

In [ ]:
print(f"Transition amplitude: {counts['0'*N_DIM]/sum(counts.values())}")

그런 다음 이 프로세스는 각 훈련 데이터 샘플 쌍에 대해 반복되어 훈련 커널 행렬을 채우고 각 훈련 데이터 샘플과 테스트 데이터 샘플 사이에 테스트 커널 행렬을 채웁니다. 각 행렬은 대칭이므로 계산 시간을 줄이기 위해 항목의 절반만 계산됩니다.

아래의 코드로 훈련 및 테스트 커널 행렬을 계산하고 그려봅시다:

In [ ]:
matrix_train = pauli_kernel.evaluate(x_vec=sample_train)
matrix_val = pauli_kernel.evaluate(x_vec=sample_val, y_vec=sample_train)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_val),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("validation kernel matrix")
plt.show()

</div>
    
<div class="alert alert-block alert-success">

<b>도전 3b</b> 

$x = (-0.5, -0.4, 0.3, 0, -0.9)$와 $y = (0, -0.7, -0.3, 0, -0.4)$ 사이의 전이 진폭을 'ZZFeatureMap'을 3회 반복,'회전' 얽힘 및 나머지는 기본값을 사용하여 계산합니다. 'shots=8192', 'seed_simulator=1024' 및 'seed_transpiler=1024'와 함께 'qasm_simulator'를 사용합니다.
    
</div>

In [ ]:
x = [-0.5, -0.4, 0.3, 0, -0.9]
y = [0, -0.7, -0.3, 0, -0.4]

##############################
# Provide your code here

ex3b_amp = 



##############################

In [ ]:
# Check your answer and submit using the following code
from qc_grader import grade_ex3b
grade_ex3b(ex3b_amp)

관련 QGSS 자료:
- [**Kernel Trick (Lecture 6.1)**](https://www.youtube.com/watch?v=m6EzmYsEOiI&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=14)
- [**Kernel Trick (Lecture 6.2)**](https://www.youtube.com/watch?v=zw3JYUrS-v8&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=15)

### 2.3 양자 서포트 벡터 머신 (QSVM)

[***Havlicek et al*.  Nature 567, 209-212 (2019)**](https://www.nature.com/articles/s41586-019-0980-2) 의 논문에서 소개된, 양자 커널 서포트 벡터 분류 알고리즘은 다음의 단계로 구성됩니다:


<center><img src="./resources/qsvc.png" width="1000"></center>

1. 학습/테스트 양자 커널 행렬들을 구성합니다.
    1. 학습 데이터넷의 모든 데이터 포인트 페어 $\mathbf{x}_{i},\mathbf{x}_j$에, 특징 맵을 적용하고 전이 확률을 측정합니다  : $ K_{ij} = \left| \langle 0 | \mathbf{U}^\dagger_{\Phi(\mathbf{x_j})} \mathbf{U}_{\Phi(\mathbf{x_i})} | 0 \rangle \right|^2 $.
    2. 모든 학습 데이터 포인트  $\mathbf{x_i}$ and testing point $\mathbf{y_j}$에, 특징 맵을 적용하고 전이 확률을 측정합니다: $ K_{ij} = \left| \langle 0 | \mathbf{U}^\dagger_{\Phi(\mathbf{y_j})} \mathbf{U}_{\Phi(\mathbf{x_i})} | 0 \rangle \right|^2 $.
2. 학습과 테스트 양자 커널 행렬을 고전적인 서포트 벡터 머신 분류 알고리즘에서 사용합니다.

`scikit-learn` `svc` 알고리즘에서 사용자 정의 커널 [**define a custom kernel**](https://scikit-learn.org/stable/modules/svm.html#custom-kernels)은 두 가지 방법으로 구성할 수 있습니다: 1) 커널을 호출 가능한 함수로 제공, 2) 커널 행렬을 미리 계산하여 제공. Qiskit의  `QuantumKernel` 클래스를 사용하여 이 중 하나를 수행할 수 있습니다.


아래의 코드는 앞서 계산한 학습과 테스트 커널 행렬을 가져온 후 `scikit-learn` `svc`에 전달합니다. 

In [ ]:
pauli_svc = SVC(kernel='precomputed')
pauli_svc.fit(matrix_train, labels_train)
pauli_score = pauli_svc.score(matrix_val, labels_val)

print(f'Precomputed kernel classification test score: {pauli_score*100}%')

관련 QGSS 자료:
- [**Classical SVM (Lecture 4.2)**](https://www.youtube.com/watch?v=lpPij21jnZ4&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=9)
- [**Quantum Classifier (Lecture 5.1)**](https://www.youtube.com/watch?v=-sxlXNz7ZxU&list=PLOFEBzvs-VvqJwybFxkTiDzhf5E11p8BI&index=11)

## Part 2: 도전 - Fashion-MNIST의 3-클래스 분류 QSVM


이 파트에서는 지금까지 배운 내용을 활용하여 의류 이미지의 3 클래스 분류를 구현하고 정확도를 향상 시켜보겠습니다. 

</div>
    
<div class="alert alert-block alert-success">

<b>도전 3c</b> 

**목표**: QSVM을 사용하여 3 클래스 분류기를 구현하고 더 작은 특징 맵 회로를 사용하여 의류 이미지 데이터 세트에서 75% 정확도를 달성합니다.

**데이터세트**: Fashion-MNIST 의류 이미지 데이터세트. 이 챌린지 연습에는 다음 세 가지 데이터 세트가 있습니다.
- 학습: 이미지와 라벨이 주어집니다.
- 공개 테스트: 이미지는 주어지지만 라벨은 숨겨져 있습니다. 
- 비공개 테스트: 이미지와 라벨 모두 숨겨져 있습니다.
    
평가는 공개 테스트 및 비공개 테스트 데이터 모두에 대해 진행되며 그 이유는 양자적 방법이 사용된 학습은 부정 행위가 불가능하다는 것을 보여주기 위함입니다. 
    
</div>

### 이진 분류기를 사용하여 다중 클래스 분류기를 구현하기


지금까지 QSVM으로 이진 분류를 구현하는 방법을 배웠습니다. 이제 어떻게 이것을 다중 클래스 분류로 확장할 수 있을까요? 두 가지 방법이 있습니다. 하나는 일대다(One-vs-Rest) 접근 방식이고 다른 하나는 일대일(One-vs-One) 접근 방식입니다.

1. 일대다 (One-vs-Rest): 이 접근 방식에서 다중 클래스 분류는 대상 클래스를 양수로 분류하고 나머지는 음수로 분류하는 각 클래스에 대한 분류기를 결합하여 이루어집니다. 각 클래스마다 하나의 분류기가 필요하기 때문에 N-class 분류에 필요한 총 분류기의 수는 N입니다. 필요한 분류기가 더 적은 것이 장점이고, 각 분류 과정에서 레이블이 불균형하기 쉽다는 단점이 있습니다.
2. 일대일 (One-vs-One): 이 접근 방식에서 다중 클래스 분류는 두 클래스의 각 쌍에 대해 분류기를 결합하여 이루어지며, 여기서 하나는 양수이고 다른 하나는 음수입니다. 각 레이블 쌍에 대해 하나의 분류기가 필요하므로 N 클래스 분류에 필요한 총 분류기 수는 N(N-1)/2입니다. 장점은 레이블이 각 분류에서 불균형할 가능성이 적고, 단점은 필요한 분류기의 수가 더 많다는 것입니다.

두 가지 접근 방식 모두 문제를 해결하는 데 사용할 수 있지만 여기서는 일대다 (One-vs-Rest) 접근 방식을 기반으로 한 힌트가 제공됩니다. 힌트를 따라 문제를 해결해 나가십시오.

<div><img src="./resources/onevsrest.png" width="800"></div>

그림 출처: [cc.gatech.edu](https://www.cc.gatech.edu/classes/AY2016/cs4476_fall/results/proj4/html/jnanda3/index.html)

### 1. 데이터 준비
작업의 대상이 되는 데이터는 패션-MNIST라는 의류 이미지 데이터 세트의 작은 부분 집합으로, MNIST 데이터 세트의 변형입니다. 아래와 같은 라벨을 분류하는 것을 목표로 합니다.
- label 0: T-shirt/top
- label 2: pullover
- label 3: dress

우선, 데이터 세트를 로드하고 각 클래스의 이미지를 하나씩 화면에 띄워 보겠습니다.

In [ ]:
# Load MNIST dataset
DATA_PATH = './resources/ch3_part2.npz'
data = np.load(DATA_PATH)

sample_train = data['sample_train']
labels_train = data['labels_train']
sample_test = data['sample_test']

# Split train data
sample_train, sample_val, labels_train, labels_val = train_test_split(
    sample_train, labels_train, test_size=0.2, random_state=42)

# Visualize samples
fig = plt.figure()

LABELS = [0, 2, 3]
num_labels = len(LABELS)
for i in range(num_labels):
    ax = fig.add_subplot(1, num_labels, i+1)
    img = sample_train[labels_train==LABELS[i]][0].reshape((28, 28))
    ax.imshow(img, cmap="Greys")

그런 다음 이전과 동일한 방식으로 데이터 세트를 전처리합니다.
- 표준화
- PCA
- 정규화

여기서 N_DIM을 변경하여 특징의 수를 변경할 수 있습니다.

In [ ]:
# Standardize
standard_scaler = StandardScaler()
sample_train = standard_scaler.fit_transform(sample_train)
sample_val = standard_scaler.transform(sample_val)
sample_test = standard_scaler.transform(sample_test)

# Reduce dimensions
N_DIM = 5
pca = PCA(n_components=N_DIM)
sample_train = pca.fit_transform(sample_train)
sample_val = pca.transform(sample_val)
sample_test = pca.transform(sample_test)

# Normalize
min_max_scaler = MinMaxScaler((-1, 1))
sample_train = min_max_scaler.fit_transform(sample_train)
sample_val = min_max_scaler.transform(sample_val)
sample_test = min_max_scaler.transform(sample_test)

### 2. 모델링
One-vs-Rest 접근 방식을 기반으로 다음 세 가지 QSVM 바이너리 분류기를 생성해야 합니다.
- 라벨 0 대 나머지
- 라벨 2 대 나머지
- 라벨 3 대 나머지

첫번째 분류기를 힌트로 제공합니다.

### 2.1: 라벨 0 대 나머지
다음과 같이 라벨 0이 양수(1)이고 나머지가 음수(0)인 새 라벨을 만듭니다.

In [ ]:
labels_train_0 = np.where(labels_train==0, 1, 0)
labels_val_0 = np.where(labels_val==0, 1, 0)

print(f'Original validation labels:      {labels_val}')
print(f'Validation labels for 0 vs Rest: {labels_val_0}')

원래 라벨이 0이었던 곳만 1로 바뀌었습니다.

다음으로 이전과 같이 QSVM을 사용하여 이진 분류기를 구성합니다.
이 힌트에서는 PauliFeatureMap을 사용하지만 다른 특징 맵도 사용할 수 있습니다.

In [ ]:
pauli_map_0 = PauliFeatureMap(feature_dimension=N_DIM, reps=2, paulis = ['X', 'Y', 'ZZ'])
pauli_kernel_0 = QuantumKernel(feature_map=pauli_map_0, quantum_instance=Aer.get_backend('statevector_simulator'))

pauli_svc_0 = SVC(kernel='precomputed', probability=True)

matrix_train_0 = pauli_kernel_0.evaluate(x_vec=sample_train)
pauli_svc_0.fit(matrix_train_0, labels_train_0)

matrix_val_0 = pauli_kernel_0.evaluate(x_vec=sample_val, y_vec=sample_train)
pauli_score_0 = pauli_svc_0.score(matrix_val_0, labels_val_0)
print(f'Accuracy of discriminating between label 0 and others: {pauli_score_0*100}%')

QSVM 바이너리 분류기가 라벨 0과 나머지를 괜찮은 확률로 구별할 수 있음을 알 수 있습니다.

마지막으로 각 테스트 데이터에 대해 라벨 0일 확률을 계산합니다. ```predict_proba``` 메소드로 계산할 수 있습니다.

In [ ]:
matrix_test_0 = pauli_kernel_0.evaluate(x_vec=sample_test, y_vec=sample_train)
pred_0 = pauli_svc_0.predict_proba(matrix_test_0)[:, 1]
print(f'Probability of label 0: {np.round(pred_0, 2)}')

이 확률은 다중 클래스 분류에서 중요한 지표입니다.
같은 방법으로 나머지 두 라벨에 대한 확률을 구합니다.

### 2.2: 라벨 2 vs 나머지
QSVM을 사용하여 이진 분류기를 만들고 테스트 데이터 세트에 대한 라벨 2의 확률을 얻습니다.

In [ ]:
##############################
# Provide your code here

pred_2 = 


##############################

### 2.3 라벨 3 vs Rest
QSVM을 사용하여 이진 분류기를 만들고 테스트 데이터 세트에 대한 라벨 3의 확률을 얻습니다.

In [ ]:
##############################
# Provide your code here

pred_3 = 


##############################

### 3. 예측
마지막으로 각 레이블의 확률을 기반으로 최종 예측을 수행합니다.
제출하는 예측값은 다음 형식이어야 합니다.

In [ ]:
sample_pred = np.load('./resources/ch3_part2_sub.npy')
print(f'Sample prediction: {sample_pred}')

다중 클래스 분류에 대한 예측을 수행하는 방법을 이해하기 위해 라벨 2와 라벨 3의 두 라벨에 대해 예측하는 경우부터 살펴봅시다.

특정 데이터에 대한 확률이 다음과 같으면 라벨 2가 가장 높게 예측되어야 합니다.
- 라벨 2의 확률: 0.7
- 라벨 3의 확률: 0.2

```np.where``` 함수로 이것을 구현할 수 있습니다. (물론 다른 방법을 사용해도 됩니다.)

In [ ]:
pred_2_ex = np.array([0.7])
pred_3_ex = np.array([0.2])

pred_test_ex = np.where((pred_2_ex > pred_3_ex), 2, 3)
print(f'Prediction: {pred_test_ex}')

이 방법을 그대로 여러 데이터 문제에 적용할 수 있습니다.

두 번째 데이터가 각 라벨에 대해 다음과 같은 확률을 가질 경우 라벨 3으로 분류하도록 합니다.
- 라벨 2의 확률: 0.1
- 라벨 3의 확률: 0.6

In [ ]:
pred_2_ex = np.array([0.7, 0.1])
pred_3_ex = np.array([0.2, 0.6])

pred_test_ex = np.where((pred_2_ex > pred_3_ex), 2, 3)
print(f'Prediction: {pred_test_ex}')

이 방법을 확장하여 3-클래스 분류에 대한 예측을 할 수 있습니다.

힌트로 주어진 방법을 확장해 구현하고 최종 3-클래스 예측을 수행합니다.

In [ ]:
##############################
# Provide your code here

pred_test = 


##############################

### 4. 제출
</div>
    
<div class="alert alert-block alert-success">

<b>Challenge 3c</b> 

**제출**: 다음 11개의 아이템을 제출합니다.
- **pred_test**: 공개 테스트 데이터 세트에 대한 예측
- **sample_train**: 커널을 얻는 데 사용되는 학습 데이터
- **standard_scaler**: 데이터를 표준화하는 데 사용되는 것
- **pca**: 차원을 줄이는 데 사용되는 것
- **min_max_scaler**: 데이터를 정규화하는 데 사용되는 것
- **kernel_0**: "label 0 vs rest" 분류기용 커널
- **kernel_2**: "label 2 vs rest" 분류기용 커널
- **kernel_3**: "label 3 vs rest" 분류기용 커널
- **svc_0**: "label 0 vs rest"를 분류하도록 훈련된 SVC
- **svc_2**: "label 2 vs rest"를 분류하도록 훈련된 SVC
- **svc_3**: "label 3 vs rest"를 분류하도록 훈련된 SVC

**기준**: 공개 및 비공개 테스트 데이터 모두에서 70% 이상의 정확도.

**점수**: 기준을 통과한 솔루션은 다음과 같이 점수가 매겨집니다. 이 최종 점수는 작을수록 좋습니다.
1. 각 특징 맵은 다음과 같이 변환됩니다.
    - basis_gates=['u1', 'u2', 'u3', 'cx']
    - optimization_level=0
2. 각 트랜스파일된 회로의 비용을 계산합니다.
    비용 = 10 * #cx + (#u1 + #u2 + #u3)
3. 비용의 합이 최종 점수가 됩니다.

</div>

정리하자면, 다음의 것들이 제출 되어야 합니다.
- 공개 테스트 데이터에 대한 예측: (**sample_test**)
- type: numpy.ndarray
- shape: (20,)

In [ ]:
print(f'Sample prediction: {sample_pred}')

In [ ]:
# Submit your answer. You can re-submit at any time.
from qc_grader import grade_ex3c

grade_ex3c(pred_test, sample_train, 
           standard_scaler, pca, min_max_scaler,
           kernel_0, kernel_2, kernel_3,
           svc_0, svc_2, svc_3)

## 추가 정보

**제작자:** Shota Nakasuji, Anna Phan

**한글번역:** 신소영, 김정원

**Version:** 1.0.0